In [ ]:
# Installations to run in Planetary Computer environment 
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import geopandas as gpd

In [ ]:
## specify areas of interest / districts and metadata
## URL method accessed an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("geo_name") to create properties for output file
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-CRI-San_Jose-ADM2.geojson'
#'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-MAR-Marrakech-ADM1.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)

#URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/urban_edge_t3.geojson'
#DistrictsGDF = gpd.read_file(URL)
#DistrictsGDF.sample(3)
#Districts = geemap.gdf_to_ee(DistrictsGDF)

#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
#Districts = ee.FeatureCollection('projects/wri-datalab/AUE/urban_edge/urban_edge_t3').first()

cityname = os.path.splitext(os.path.basename(URL))[0].split('-',2)[2].rsplit('-',1)[0]
def Rename(feat):
    return feat.set('geo_name',cityname)
#Districts = Districts.union(1).map(Rename)

DistrictsProjCRS = Districts.geometry().projection().crs()
print(DistrictsProjCRS.getInfo())
print(Districts.first().getString('geo_name').getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]

Areaofinterest = AOIname ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
print(Areaofinterest)

In [ ]:
## create map
Map = geemap.Map(height="350px")
#Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)

In [ ]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection();  
print('WorldCover projection:', WCprojection.getInfo());

Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

In [ ]:
## Add intra-urban land use dataset

ULU = ee.ImageCollection("projects/wri-datalab/urban_land_use/v1")

WRIulu = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIulu = WRIulu.mask(WRIulu.mask().gt(0))
WRIroad = ULU.select('road_lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIuluwRoad = WRIulu.add(WRIroad).where(WRIroad.eq(1),6).mask(WRIulu.mask().gt(0))

ULUmaskedESA = WRIuluwRoad.updateMask(WorldCover.eq(50)) #.Or(WorldCover.eq(60)))

ULUmaskedESA = ULUmaskedESA.reproject(
      crs= WCprojection
    )

CLASSES_7=[
  "open_space",
  "nonresidential",
  "atomistic",
  "informal_subdivision",
  "formal_subdivision",
  "housing_project",
  "road"]
COLORS_7=[
  '33A02C',
  'E31A1C',
  'FB9A99',
  'FFFF99',
  '1F78B4',
  'A6CEE3',
  '3f3f3f']  
ULU7Params = {"bands": ['lulc'], 'min': 0, 'max': 6, "opacity": 1, "palette": COLORS_7}

Map.addLayer(ULUmaskedESA,ULU7Params,"Urban Land Use 2020 (WRI) masked to WorldCover built",False)

In [ ]:
# set geometries and date range of interest for land surface temperature (LST) calculation

roi = Districts
ROIcenter = roi.geometry().centroid(1)

date_start = '2020-01-01'
date_end = '2021-01-01'
image_limit = 50 # max number of images to include, sorted from least to most cloudy


In [ ]:

## CONFIG

S2 = ee.ImageCollection("COPERNICUS/S2_SR")
S2C = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

MAX_CLOUD_PROB=30
S2_ALBEDO_EQN='((B*Bw)+(G*Gw)+(R*Rw)+(NIR*NIRw)+(SWIR1*SWIR1w)+(SWIR2*SWIR2w))'
S2_VIZ = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3};


## METHODS

def mask_clouds_and_rescale(im):
    clouds=ee.Image(im.get('cloud_mask')).select('probability')
    return im.updateMask(clouds.lt(MAX_CLOUD_PROB)).divide(10000)

def get_masked_s2_collection(roi,start,end):
  criteria=(ee.Filter.And(
            ee.Filter.date(start,end),
            ee.Filter.bounds(roi)
        ))
  s2=S2.filter(criteria).limit(image_limit,'CLOUDY_PIXEL_PERCENTAGE')#.select('B2','B3','B4','B8','B11','B12')
  s2c=S2C.filter(criteria).limit(image_limit,'CLOUDY_PIXEL_PERCENTAGE')
  joined=(ee.Join.saveFirst('cloud_mask').apply(**{
        'primary': ee.ImageCollection(s2),
        'secondary': ee.ImageCollection(s2c),
        'condition': ee.Filter.equals(**{'leftField':'system:index','rightField':'system:index'}) 
  }))
  cloudless=ee.ImageCollection(joined).map(mask_clouds_and_rescale)
  return cloudless

# weights derived from 
# S. Bonafoni and A. Sekertekin, "Albedo Retrieval From Sentinel-2 by New Narrow-to-Broadband Conversion Coefficients," in IEEE Geoscience and Remote Sensing Letters, vol. 17, no. 9, pp. 1618-1622, Sept. 2020, doi: 10.1109/LGRS.2020.2967085.
def calc_s2_albedo(image):
  config={
    'Bw':0.2266,
    'Gw':0.1236,
    'Rw':0.1573,
    'NIRw':0.3417,
    'SWIR1w':0.1170,
    'SWIR2w':0.0338,
    'B':image.select('B2'),
    'G':image.select('B3'),
    'R':image.select('B4'),
    'NIR':image.select('B8'),
    'SWIR1':image.select('B11'),
    'SWIR2':image.select('B12')
  }
  return image.expression(S2_ALBEDO_EQN,config).double().rename('albedo')



## S2 MOSAIC AND ALBEDO

dataset = get_masked_s2_collection(roi,date_start,date_end)
s2_albedo = dataset.map(calc_s2_albedo)
mosaic=dataset.mean()
#albedoMean=s2_albedo.mean()
albedoMean=s2_albedo.reduce(ee.Reducer.mean())

#print(dataset.size().getInfo())
#print(albedoMean.getInfo())
#print(albedoMean.projection().nominalScale().getInfo())

In [ ]:
# define "low albedo" threshold
LowAlbedoMax = 0.20 # EnergyStar steep slope minimum initial value is 0.25. 3-year value is 0.15. https://www.energystar.gov/products/building_products/roof_products/key_product_criteria
albedoMeanThres = albedoMean.updateMask(albedoMean.lt(LowAlbedoMax))

#Add S2 albedo means to map

Map.addLayer(mosaic, S2_VIZ, 'S2',False)
Map.addLayer(albedoMean, {'min':0, 'max':1}, 'S2 albedo',False)
Map.addLayer(albedoMeanThres,{'min':0, 'max':1}, 'S2 albedo, areas below '+str(LowAlbedoMax)+' albedo',True)

Map.addLayer(roi,{}, "Area of interest",False,0.3)
Map

In [ ]:
## calculations to determine Albedo by LULC class


## function to create image of means of toCount for each asClass

def getmeanbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('') #'class_count-'
                                       .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass
def getcountbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_count-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass filtered by toCount
def getcountbyclassFilt(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue).And(toCount.gte(0))) # uncomment And statement if you want include only pixels that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_countFilt-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))

In [ ]:
## create image with each WorldCover class mean as a band

asClass = WorldCover
toCount = albedoMean 

meanbyclass=ee.Image(getmeanbyclass(10)).addBands([
  getmeanbyclass(20),  
  getmeanbyclass(30),  
  getmeanbyclass(40),  
  getmeanbyclass(50),  
  getmeanbyclass(60),
  getmeanbyclass(70),  
    getmeanbyclass(80), 
    getmeanbyclass(90), 
    getmeanbyclass(95), 
    getmeanbyclass(100), 
])

## create image with each WorldCover class count as a band

countbyclass=ee.Image(getcountbyclass(10)).addBands([
  getcountbyclass(20),  
  getcountbyclass(30),  
  getcountbyclass(40),  
  getcountbyclass(50),  
  getcountbyclass(60),
  getcountbyclass(70),  
    getcountbyclass(80), 
    getcountbyclass(90), 
    getcountbyclass(95), 
    getcountbyclass(100), 
])

## create image with each WorldCover class count above LST threshold as a band
toCount = albedoMeanThres 

countbyclassFilt=ee.Image(getcountbyclassFilt(10)).addBands([
  getcountbyclassFilt(20),  
  getcountbyclassFilt(30),  
  getcountbyclassFilt(40),  
  getcountbyclassFilt(50),  
  getcountbyclassFilt(60),
  getcountbyclassFilt(70),  
    getcountbyclassFilt(80), 
    getcountbyclassFilt(90), 
    getcountbyclassFilt(95), 
    getcountbyclassFilt(100), 
])

#print('meanbyclass', meanbyclass.getInfo())
#print('countbyclass', countbyclass.getInfo())
#print('countbyclassFilt', countbyclassFilt.getInfo())

#Map.addLayer(meanbyclass.select('50'),{},"meanbyWCclass")
#Map.addLayer(countbyclassFilt.select('class_countFilt-10'),{},"countbyFiltWCclass")
#Map.addLayer(countbyclass.select('class_count-10'),{},"countbyWCclass")

In [ ]:
## create image with each ULU class mean as a band

asClass = ULUmaskedESA
toCount = albedoMean 

meanbyclassULU=ee.Image(getmeanbyclass(0)).addBands([
  getmeanbyclass(1),  
  getmeanbyclass(2),  
  getmeanbyclass(3),  
  getmeanbyclass(4),  
  getmeanbyclass(5),
  getmeanbyclass(6) 
])

## create image with each ULU class count as a band

countbyclassULU=ee.Image(getcountbyclass(0)).addBands([
  getcountbyclass(1),  
  getcountbyclass(2),  
  getcountbyclass(3),  
  getcountbyclass(4),  
  getcountbyclass(5),
  getcountbyclass(6)
])

## create image with each WorldCover class count above LST threshold as a band
toCount = albedoMeanThres 

countbyclassFiltULU=ee.Image(getcountbyclassFilt(0)).addBands([
  getcountbyclassFilt(1),  
  getcountbyclassFilt(2),  
  getcountbyclassFilt(3),  
  getcountbyclassFilt(4),  
  getcountbyclassFilt(5),
  getcountbyclassFilt(6)
])

#Map.addLayer(meanbyclassULU.select('1'),{},"meanbyULUclass")
#Map.addLayer(countbyclassFiltULU.select('class_countFilt-0'),{},"countbyFiltULUclass")
#Map.addLayer(countbyclassULU.select('class_count-0'),{},"countbyULUclass")

In [ ]:
# ## create FeatureCollection with mean of count for each class for each feature

histo=meanbyclass.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 4
)

histo=countbyclass.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 4
)

histo=countbyclassFilt.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 4
)

histo=meanbyclassULU.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= histo, 
  scale= 10, 
  tileScale= 4
)

histo=countbyclassULU.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 4
)

histo=countbyclassFiltULU.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 4
)

#print('histo:', histo.limit(1,'class_count-10',False).getInfo())
#print('histo:', histo.first().toDictionary().getInfo())

In [ ]:
## Define function to normalize count as percent of all pixels in each feature and create new properties with the values

def count_to_percent(feat):
    feat=ee.Feature(feat)
    hist=ee.Dictionary(feat.toDictionary(['10','20','30','40','50','60','70','80','90','95','100']))
    hist=hist.set('10',hist.get('10',0))
    hist=hist.set('20',hist.get('20',0))
    hist=hist.set('30',hist.get('30',0))
    hist=hist.set('40',hist.get('40',0))
    hist=hist.set('50',hist.get('50',0))
    hist=hist.set('60',hist.get('60',0))
    hist=hist.set('70',hist.get('70',0))
    hist=hist.set('80',hist.get('80',0))
    hist=hist.set('90',hist.get('90',0))
    hist=hist.set('95',hist.get('95',0))
    hist=hist.set('100',hist.get('100',0))
    
    def pct_hist(k,v):
        # convert whole number (0-100) to decimal percent (0-1)
        return ee.Number(v)
    
    meansLULC = hist.map(pct_hist)
    
    histC=ee.Dictionary(feat.toDictionary(['class_count-10','class_count-20','class_count-30','class_count-40','class_count-50','class_count-60','class_count-70','class_count-80','class_count-90','class_count-95','class_count-100']))
    histC=histC.set('10',histC.get('class_count-10',0))
    histC=histC.set('20',histC.get('class_count-20',0))
    histC=histC.set('30',histC.get('class_count-30',0))
    histC=histC.set('40',histC.get('class_count-40',0))
    histC=histC.set('50',histC.get('class_count-50',0))
    histC=histC.set('60',histC.get('class_count-60',0))
    histC=histC.set('70',histC.get('class_count-70',0))
    histC=histC.set('80',histC.get('class_count-80',0))
    histC=histC.set('90',histC.get('class_count-90',0))
    histC=histC.set('95',histC.get('class_count-95',0))
    histC=histC.set('100',histC.get('class_count-100',0))
    
    histCfilt=ee.Dictionary(feat.toDictionary(['class_countFilt-10','class_countFilt-20','class_countFilt-30','class_countFilt-40','class_countFilt-50','class_countFilt-60','class_countFilt-70','class_countFilt-80','class_countFilt-90','class_countFilt-95','class_countFilt-100']))
    histCfilt=histCfilt.set('10',histCfilt.get('class_countFilt-10',0))
    histCfilt=histCfilt.set('20',histCfilt.get('class_countFilt-20',0))
    histCfilt=histCfilt.set('30',histCfilt.get('class_countFilt-30',0))
    histCfilt=histCfilt.set('40',histCfilt.get('class_countFilt-40',0))
    histCfilt=histCfilt.set('50',histCfilt.get('class_countFilt-50',0))
    histCfilt=histCfilt.set('60',histCfilt.get('class_countFilt-60',0))
    histCfilt=histCfilt.set('70',histCfilt.get('class_countFilt-70',0))
    histCfilt=histCfilt.set('80',histCfilt.get('class_countFilt-80',0))
    histCfilt=histCfilt.set('90',histCfilt.get('class_countFilt-90',0))
    histCfilt=histCfilt.set('95',histCfilt.get('class_countFilt-95',0))
    histCfilt=histCfilt.set('100',histCfilt.get('class_countFilt-100',0))
    
    def area_hist(k,v):
        # convert 10m pixel count of class to KM2 of class
        return ee.Number(v).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    
    classAreas = histC.map(area_hist)
    classFiltAreas = histCfilt.map(area_hist)

    
    FeatArea = feat.area(0.001).multiply(0.000001)
    cityID = Areaofinterest
    geo_level = feat.getString("geo_level")
    geo_name = feat.getString("geo_name").split(' ').join('_')
    #geo_name = feat.getString("Sub_City").cat(ee.String("-")).cat(feat.getString("Woreda"))
    #geo_name = feat.getString("city_name_viz").split(' ').join('_')
    #geo_name = feat.getString("City Name")
    geo_id = ee.String(cityID+"-").cat(geo_name)
    source = "Sentinel-2, ESA WorldCover, WRI ULU"

    totalPixels=hist.values()
    
    histULU=ee.Dictionary(feat.toDictionary(['0','1','2','3','4','5','6']))
    histULU=histULU.set('0',histULU.get('0',0))
    histULU=histULU.set('1',histULU.get('1',0))
    histULU=histULU.set('2',histULU.get('2',0))
    histULU=histULU.set('3',histULU.get('3',0))
    histULU=histULU.set('4',histULU.get('4',0))
    histULU=histULU.set('5',histULU.get('5',0))
    histULU=histULU.set('6',histULU.get('6',0))
    
    meansULU = histULU.map(pct_hist)
        
    histULUc=ee.Dictionary(feat.toDictionary(['class_count-0','class_count-1','class_count-2','class_count-3','class_count-4','class_count-5','class_count-6']))
    histULUc=histULUc.set('0',histULUc.get('class_count-0',0))
    histULUc=histULUc.set('1',histULUc.get('class_count-1',0))
    histULUc=histULUc.set('2',histULUc.get('class_count-2',0))
    histULUc=histULUc.set('3',histULUc.get('class_count-3',0))
    histULUc=histULUc.set('4',histULUc.get('class_count-4',0))
    histULUc=histULUc.set('5',histULUc.get('class_count-5',0))
    histULUc=histULUc.set('6',histULUc.get('class_count-6',0))
    
    histULUcFilt=ee.Dictionary(feat.toDictionary(['class_countFilt-0','class_countFilt-1','class_countFilt-2','class_countFilt-3','class_countFilt-4','class_countFilt-5','class_countFilt-6']))
    histULUcFilt=histULUcFilt.set('0',histULUcFilt.get('class_countFilt-0',0))
    histULUcFilt=histULUcFilt.set('1',histULUcFilt.get('class_countFilt-1',0))
    histULUcFilt=histULUcFilt.set('2',histULUcFilt.get('class_countFilt-2',0))
    histULUcFilt=histULUcFilt.set('3',histULUcFilt.get('class_countFilt-3',0))
    histULUcFilt=histULUcFilt.set('4',histULUcFilt.get('class_countFilt-4',0))
    histULUcFilt=histULUcFilt.set('5',histULUcFilt.get('class_countFilt-5',0))
    histULUcFilt=histULUcFilt.set('6',histULUcFilt.get('class_countFilt-6',0))

    classAreasULU = histULUc.map(area_hist)
    classFiltAreasULU = histULUcFilt.map(area_hist)

    return feat.set({
        'LC10albedo': meansLULC.getNumber('10'),
        'LC20albedo': meansLULC.getNumber('20'),
        'LC30albedo': meansLULC.getNumber('30'),
        'LC40albedo': meansLULC.getNumber('40'),
        'LC50albedo': meansLULC.getNumber('50'),
        'LC60albedo': meansLULC.getNumber('60'),
        'LC70albedo': meansLULC.getNumber('70'),
        'LC80albedo': meansLULC.getNumber('80'),
        'LC90albedo': meansLULC.getNumber('90'),
        'LC95albedo': meansLULC.getNumber('95'),
        'LC100albedo': meansLULC.getNumber('100'),
        'TotalareaKM2': FeatArea,
        'TotalPixels': totalPixels,
        'geo_level': geo_level,
        'geo_name': geo_name,
        'geo_id': geo_id,
        'date_start': date_start,
        'date_end': date_end,
        'source':source,
        'LC10areaKM2': classAreas.getNumber('10'),
        'LC20areaKM2': classAreas.getNumber('20'),
        'LC30areaKM2': classAreas.getNumber('30'),
        'LC40areaKM2': classAreas.getNumber('40'),
        'LC50areaKM2': classAreas.getNumber('50'),
        'LC60areaKM2': classAreas.getNumber('60'),
        'LC70areaKM2': classAreas.getNumber('70'),
        'LC80areaKM2': classAreas.getNumber('80'),
        'LC90areaKM2': classAreas.getNumber('90'),
        'LC95areaKM2': classAreas.getNumber('95'),
        'LC100areaKM2': classAreas.getNumber('100'),
        'LC10lowAlbedoPct': classFiltAreas.getNumber('10').divide(classAreas.getNumber('10')),
        'LC20lowAlbedoPct': classFiltAreas.getNumber('20').divide(classAreas.getNumber('20')),
        'LC30lowAlbedoPct': classFiltAreas.getNumber('30').divide(classAreas.getNumber('30')),
        'LC40lowAlbedoPct': classFiltAreas.getNumber('40').divide(classAreas.getNumber('40')),
        'LC50lowAlbedoPct': classFiltAreas.getNumber('50').divide(classAreas.getNumber('50')),
        'LC60lowAlbedoPct': classFiltAreas.getNumber('60').divide(classAreas.getNumber('60')),
        'LC70lowAlbedoPct': classFiltAreas.getNumber('70').divide(classAreas.getNumber('70')),
        'LC80lowAlbedoPct': classFiltAreas.getNumber('80').divide(classAreas.getNumber('80')),
        'LC90lowAlbedoPct': classFiltAreas.getNumber('90').divide(classAreas.getNumber('90')),
        'LC95lowAlbedoPct': classFiltAreas.getNumber('95').divide(classAreas.getNumber('95')),
        'LC100lowAlbedoPct': classFiltAreas.getNumber('100').divide(classAreas.getNumber('100')),
        'ULU0albedo': meansULU.getNumber('0'),
        'ULU1albedo': meansULU.getNumber('1'),
        'ULU2albedo': meansULU.getNumber('2'),
        'ULU3albedo': meansULU.getNumber('3'),
        'ULU4albedo': meansULU.getNumber('4'),
        'ULU5albedo': meansULU.getNumber('5'),
        'ULU6albedo': meansULU.getNumber('6'),
        'ULU0areaKM2': classAreasULU.getNumber('0'),
        'ULU1areaKM2': classAreasULU.getNumber('1'),
        'ULU2areaKM2': classAreasULU.getNumber('2'),
        'ULU3areaKM2': classAreasULU.getNumber('3'),
        'ULU4areaKM2': classAreasULU.getNumber('4'),
        'ULU5areaKM2': classAreasULU.getNumber('5'),
        'ULU6areaKM2': classAreasULU.getNumber('6'),
        'ULU0lowAlbedoPct': classFiltAreasULU.getNumber('0').divide(classAreasULU.getNumber('0')),
        'ULU1lowAlbedoPct': classFiltAreasULU.getNumber('1').divide(classAreasULU.getNumber('1')),
        'ULU2lowAlbedoPct': classFiltAreasULU.getNumber('2').divide(classAreasULU.getNumber('2')),
        'ULU3lowAlbedoPct': classFiltAreasULU.getNumber('3').divide(classAreasULU.getNumber('3')),
        'ULU4lowAlbedoPct': classFiltAreasULU.getNumber('4').divide(classAreasULU.getNumber('4')),
        'ULU5lowAlbedoPct': classFiltAreasULU.getNumber('5').divide(classAreasULU.getNumber('5')),
        'ULU6lowAlbedoPct': classFiltAreasULU.getNumber('6').divide(classAreasULU.getNumber('6')),
    })

In [ ]:
## update FeatureCollection with percents

albedo_means=histo.map(count_to_percent)

#print('LST stats by Land Cover class for Districts',albedo_means.limit(1).getInfo());
#print('Albedo stats by Land Cover class for Districts',albedo_means.first().toDictionary().getInfo())

In [ ]:
## render on map percent tree cover by class from feature collection

empty = ee.Image().byte()
Tpctfills = empty.paint(**{'featureCollection': albedo_means,'color': 'LC50lowAlbedoPct'})

fillspalette = ['green', 'red']
cmap1 = ['blue', 'cyan', 'green', 'yellow', 'red']
#Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':1}, '% of built areas with low albedo', True, 0.65)
#Map

In [ ]:
## select properties to keep, sort features and create data frame to display properties
albedo_meansSort = albedo_means.select([
    'TotalareaKM2',
    'geo_level',
    'geo_name',
    'geo_id',
    'date_start',
    'date_end',
    'source',
    'LC10albedo','LC20albedo','LC30albedo','LC40albedo','LC50albedo','LC60albedo','LC70albedo','LC80albedo','LC90albedo','LC95albedo','LC100albedo',
    'ULU0albedo','ULU1albedo','ULU2albedo','ULU3albedo','ULU4albedo','ULU5albedo','ULU6albedo',
    'LC10lowAlbedoPct','LC20lowAlbedoPct','LC30lowAlbedoPct','LC40lowAlbedoPct','LC50lowAlbedoPct','LC60lowAlbedoPct','LC70lowAlbedoPct','LC80lowAlbedoPct','LC90lowAlbedoPct','LC95lowAlbedoPct','LC100lowAlbedoPct',
    'ULU0lowAlbedoPct','ULU1lowAlbedoPct','ULU2lowAlbedoPct','ULU3lowAlbedoPct','ULU4lowAlbedoPct','ULU5lowAlbedoPct','ULU6lowAlbedoPct',
]).sort('LC50lowAlbedoPct', False) #
#print('Albedo means filtered and sorted',albedo_meansSort.limit(1).getInfo());
#print('Albedo means filtered and sorted:', albedo_meansSort.first().toDictionary().getInfo())

In [ ]:
#df = geemap.ee_to_pandas(albedo_meansSort)
#df.sample(3)

In [ ]:
# GJS = geemap.ee_to_geojson(albedo_meansSort)
# from ipyleaflet import GeoJSON
# json_layer_2 = GeoJSON(
#     data=GJS,
#     name='US States EE JSON',
#     hover_style={'fillColor': 'red', 'fillOpacity': 0.5},
# )
# Map.add_layer(json_layer_2)
# Map

In [ ]:
## display features in chart

import geemap.chart as chart

xProperty = 'geo_name' 
yProperties = ['LC50lowAlbedoPct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "Percent of built area with albedo less than "+str(LowAlbedoMax)+"",
    "legend_location": "top-right",
    "height": "500px",
}

#chart.feature_byFeature(albedo_meansSort, xProperty, yProperties, **options)

In [ ]:
# Download FeatureCollection of OpenBuiltAreaPct as shapefile to Google Drive

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'GeoJSON',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}
#print('Exporting {}'.format(OpenBuiltAreaPct))
task = ee.batch.Export.table.toDrive(albedo_meansSort, 'albedo_meansSort', **task_config)
task.start()